In [ ]:
#https://medium.com/mlearning-ai/general-approach-for-custom-object-detection-with-tensorflow-2-61593a67a02d

In [2]:
import os
import shutil

In [3]:
import tensorflow
print(tensorflow.__version__)

2.6.0


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Imports

In [6]:
%cd ..

/


In [7]:
%cd /usr/local/lib/python3.7/dist-packages/tensorflow

/usr/local/lib/python3.7/dist-packages/tensorflow


In [8]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 61617, done.
remote: Counting objects: 100% (61617/61617), done.
remote: Compressing objects: 100% (18125/18125), done.
remote: Total 61617 (delta 42869), reused 61615 (delta 42867), pack-reused 0
Receiving objects: 100% (61617/61617), 574.20 MiB | 29.03 MiB/s, done.
Resolving deltas: 100% (42869/42869), done.


In [9]:
%cd models/research/

/usr/local/lib/python3.7/dist-packages/tensorflow/models/research


In [10]:
os.environ['PYTHONPATH'] += f':/usr/local/lib/python3.7/dist-packages/tensorflow/models/'
os.environ['PYTHONPATH'] += f':/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/'
os.environ['PYTHONPATH'] += f':/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/slim/'
os.environ['PYTHONPATH'] += f':/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/object_detection/'

In [11]:
!pip install tf_slim

     |████████████████████████████████| 352 kB 5.2 MB/s 


In [12]:
! echo $PYTHONPATH

/env/python:/usr/local/lib/python3.7/dist-packages/tensorflow/models/:/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/:/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/slim/:/usr/local/lib/python3.7/dist-packages/tensorflow/models/research/object_detection/


In [13]:
!protoc object_detection/protos/*.proto --python_out=.

In [14]:
!cp object_detection/packages/tf2/setup.py .

In [15]:
!python -m pip install .

Processing /usr/local/lib/python3.7/dist-packages/tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.7 MB/s 
     |████████████████████████████████| 1.8 MB 23.8 MB/s 
     |████████████████████████████████| 211 kB 48.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 37.1 MB 47 kB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 4.4 MB 24.3 MB/s 
     |█████

In [16]:
!python object_detection/builders/model_builder_tf2_test.py

2021-09-08 14:03:16.481203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 14:03:17.117524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-08 14:03:17.118765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-09-08 14:03:17.204846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [17]:
%cd ..

/usr/local/lib/python3.7/dist-packages/tensorflow/models


In [18]:
os.makedirs('annotations/xmls/train', exist_ok=True)
os.makedirs('annotations/xmls/test', exist_ok=True)
os.makedirs('images/train', exist_ok=True)
os.makedirs('images/test', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('tf_record', exist_ok=True)
os.makedirs('scripts', exist_ok=True)

In [20]:
!pip install lvis

In [21]:
!pip install tensorflow-addons

In [43]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


# 0. Setup Paths

In [22]:
root = '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/'

In [23]:
WORKSPACE_PATH = f'{root}/workspace'
SCRIPTS_PATH = f'{root}/scripts'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'

# 1. Create Label Map

In [24]:
labels = [{'name':'a', 'id':1}, {'name':'b', 'id':2}]

In [25]:
with open('./annotations' + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [68]:
%cp ./annotations/label_map.pbtxt '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/workspace/annotations'

In [26]:
images = os.listdir(IMAGE_PATH + '/colectedimages')
annotations = os.listdir(ANNOTATION_PATH)
scripts = os.listdir(SCRIPTS_PATH)

In [27]:
for img in images:
  if img.endswith('.png') or img.endswith('.jpg'):
    shutil.copyfile(IMAGE_PATH + '/colectedimages/' + img, './images/' + img)
  
  if img.endswith('.xml'):
    shutil.copyfile(IMAGE_PATH + '/colectedimages/' + img, './annotations/xmls/' + img)

In [28]:
train_images = os.listdir(f'{IMAGE_PATH}/train/')

In [29]:
train_images_names = [name.replace('.png', '').replace('.jpg', '') for name in train_images if name.endswith('.png') or name.endswith('.jpg')]

In [30]:
for img in train_images:
  if img.endswith('.png') or img.endswith('.jpg'):
    shutil.copyfile(IMAGE_PATH + '/train/' + img, './images/train/' + img)
  
  if img.endswith('.xml'):
    shutil.copyfile(IMAGE_PATH + '/train/' + img, './annotations/xmls/train/' + img)

In [31]:
test_images = os.listdir(f'{IMAGE_PATH}/test/')

In [32]:
test_images_names = [name.replace('.png', '').replace('.jpg', '') for name in test_images if name.endswith('.png') or name.endswith('.jpg')]

In [33]:
for img in test_images:
  if img.endswith('.png') or img.endswith('.jpg'):
    shutil.copyfile(IMAGE_PATH + '/test/' + img, './images/test/' + img)
  
  if img.endswith('.xml'):
    shutil.copyfile(IMAGE_PATH + '/test/' + img, './annotations/xmls/test/' + img)

In [34]:
images = os.listdir('./images')

In [35]:
with open('./annotations' + '/trainval.txt', 'w') as f:
    for img in images:
      img_name = img.replace('.png', '')
      img_name = img_name.replace('.jpg', '') 
      f.write(f'{img_name}\n')

In [36]:
for script in scripts:
  if script.endswith('.py'):
    shutil.copyfile(SCRIPTS_PATH + '/' + script, './scripts/' + script)

In [37]:
csv_from_xml_df = xml_to_csv('./annotations/xmls')

In [38]:
csv_from_xml_df.to_csv('./annotations/csv_from_xml.csv')

In [39]:
train_csv = csv_from_xml_df[csv_from_xml_df['filename'].isin(train_images_names)]

In [40]:
train_csv.to_csv('train_csv.csv')

In [41]:
test_csv = csv_from_xml_df[csv_from_xml_df['filename'].isin(test_images_names)]

In [42]:
test_csv.to_csv('test_csv.csv')

# 2. Create TF records

In [44]:
!python {'./scripts/generate_tfrecord.py'} -x{'./annotations/xmls/train'} -i{'./images/train'} -l {'./annotations/label_map.pbtxt'} -o {'tf_record/train.record'}
!python {'./scripts/generate_tfrecord.py'} -x{'./annotations/xmls/test'} -i{'./images/test'} -l {'./annotations/label_map.pbtxt'} -o {'tf_record/test.record'}

Successfully created the TFRecord file: tf_record/train.record
Successfully created the TFRecord file: tf_record/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [45]:
pretrained_model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'

In [46]:
%pwd

'/usr/local/lib/python3.7/dist-packages/tensorflow/models'

In [47]:
os.makedirs('./pretrained_model', exist_ok = True)

In [48]:
%cd pretrained_model/
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -zxf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

/usr/local/lib/python3.7/dist-packages/tensorflow/models/pretrained_model
--2021-09-08 14:20:28--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.119.128, 2607:f8b0:4001:c23::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.119.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M   161MB/s    in 1.1s    

2021-09-08 14:20:29 (161 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]



In [49]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -zxf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

--2021-09-08 14:20:32--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.119.128, 2607:f8b0:4001:c23::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.119.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M   130MB/s    in 0.3s    

2021-09-08 14:20:33 (130 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]



In [50]:
%cd ..

/usr/local/lib/python3.7/dist-packages/tensorflow/models


In [51]:
checkpoint_files_1 = ['model.ckpt.meta', 'model.ckpt.index', 'model.ckpt.data-00000-of-00001']
checkpoint_files_2 = ['checkpoint', 'ckpt-0.data-00000-of-00001', 'ckpt-0.index']

In [52]:
for file in checkpoint_files_2:
  shutil.copyfile('./pretrained_model/' + pretrained_model + '/checkpoint/' + file, './checkpoints/' + file)

In [53]:
shutil.copyfile('./pretrained_model/' + pretrained_model + '/pipeline.config', './pipeline.config')

'./pipeline.config'

In [54]:
print(os.listdir('./checkpoints'))

['checkpoint', 'ckpt-0.data-00000-of-00001', 'ckpt-0.index']


# 5. Update Config For Transfer Learning

In [55]:
%cd research/

/usr/local/lib/python3.7/dist-packages/tensorflow/models/research


In [56]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [57]:
CONFIG_PATH = '../pipeline.config'

In [58]:
%pwd

'/usr/local/lib/python3.7/dist-packages/tensorflow/models/research'

## For ssd mobilenet v2, you could remove "batch_norm_trainable: true" as this field is deprecated now.

* Go to /usr/local/lib/python3.7/dist-packages/tensorflow/models/pipeline.config and delete the line batch_norm_trainable 

In [59]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [60]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [61]:
%pwd

'/usr/local/lib/python3.7/dist-packages/tensorflow/models/research'

In [62]:
pwd = '/usr/local/lib/python3.7/dist-packages/tensorflow/models'

In [63]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.model.ssd.feature_extractor.type = "ssd_mobilenet_v2_keras"
pipeline_config.train_config.fine_tune_checkpoint = f'{pwd}/checkpoints/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = f'detection'
pipeline_config.train_config.num_steps = 1500
pipeline_config.train_input_reader.label_map_path= f'{pwd}/annotations/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [f'{pwd}/tf_record/train.record']
pipeline_config.eval_input_reader[0].label_map_path = f'{pwd}/annotations/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [f'{pwd}/tf_record/test.record']

In [64]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile('../pipeline.config', "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [65]:
%pwd

'/usr/local/lib/python3.7/dist-packages/tensorflow/models/research'

In [66]:
%cd ..

/usr/local/lib/python3.7/dist-packages/tensorflow/models


In [67]:
os.makedirs('./train', exist_ok = True)
os.makedirs('./test', exist_ok = True)

# Train

In [ ]:
%mv ./pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ ./pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoints/

In [ ]:
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
!python ./research/object_detection/model_main_tf2.py \
    --train_dir=./train \
    --pipeline_config_path=pipeline.config \
    --model_dir=./train \
    --alsologtostderr

2021-09-06 17:29:16.501218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:29:16.511919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:29:16.512758: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:29:16.514387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:29:16.515233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./train/train

In [ ]:
%rm -r ./fine_tune_model

rm: cannot remove './fine_tune_model': No such file or directory


In [ ]:
os.makedirs('./fine_tune_model', exist_ok = True)

In [ ]:
%rm -r '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/fine_tune_model/'

In [ ]:
%rm -r '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/train/'

In [ ]:
os.makedirs(f'{root}/train/train', exist_ok = True)

In [ ]:
train_folder_files = os.listdir('./train')
for file in train_folder_files:
  if file == 'train': continue
  shutil.copyfile('./train/' + file, f'{root}train/{file}')

In [ ]:
train_train_files = os.listdir('./train/train')
for file in train_train_files:
  shutil.copyfile('./train/train/' + file, f'{root}train/train/{file}')

In [ ]:
train_folder_files

['ckpt-2.data-00000-of-00001',
 'ckpt-1.data-00000-of-00001',
 'checkpoint',
 'train',
 'ckpt-1.index',
 'ckpt-2.index']

In [ ]:
!python ./research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir=./train \
    --pipeline_config_path=./pipeline.config \
    --output_directory=./fine_tune_model

2021-09-06 17:36:47.121947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:36:47.132753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:36:47.133504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:36:47.142872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-06 17:36:47.143671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [ ]:
os.makedirs('/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/fine_tune_model', exist_ok=True)

In [ ]:
%cp -r ./fine_tune_model '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/fine_tune_model/'

In [ ]:
%pwd

'/usr/local/lib/python3.7/dist-packages/tensorflow/models'

In [ ]:
%cp ./pipeline.config '/content/drive/My Drive/Fax/FINKI/Semestar-8/IIS/Project/real-time-sign-language-recognition-macedonian/'

# 7. Load Train Model From Checkpoint

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
CONFIG_PATH = f'{root}/pipeline.config'
CHECKPOINT_PATH = f'{root}/train/'
PATH_TO_SAVED_MODEL = f'{root}fine_tune_model/fine_tune_model/saved_model'

In [ ]:
ckpt_files = os.listdir(CHECKPOINT_PATH)
ckpt_files

['train',
 'ckpt-2.data-00000-of-00001',
 'ckpt-1.data-00000-of-00001',
 'checkpoint',
 'ckpt-1.index',
 'ckpt-2.index']

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-2')).expect_partial()

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# 8. Detect in Real-Time

In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import cv2 
import numpy as np
import time

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(f'{root}workspace/annotations/label_map.pbtxt')

## EXAMPLE 2

In [ ]:
#Loading the image
img=os.listdir('./images/test')
print(img)
imgs = []
for im in img:
  new_img = './images/test/' + im
  imgs.append(new_img)
print(imgs)
#list containing paths of all the images

['b.5c16aac2-d400-11eb-84e0-5cf3709ec7ac.jpg', 'b.5d4c4a0c-d400-11eb-830e-5cf3709ec7ac.jpg', 'a.4ae799c6-d400-11eb-930d-5cf3709ec7ac.jpg', 'a.4c1c8db4-d400-11eb-80f6-5cf3709ec7ac.jpg']
['./images/test/b.5c16aac2-d400-11eb-84e0-5cf3709ec7ac.jpg', './images/test/b.5d4c4a0c-d400-11eb-830e-5cf3709ec7ac.jpg', './images/test/a.4ae799c6-d400-11eb-930d-5cf3709ec7ac.jpg', './images/test/a.4c1c8db4-d400-11eb-80f6-5cf3709ec7ac.jpg']


In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline


def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
def run_inference_for_single_image(image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = detect_fn
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
image_path = imgs[2]
image_np = load_image_into_numpy_array(image_path)
#print(image_np)
input_tensor = np.expand_dims(image_np, 0)
start_time = time.time()
detections = detect_fn(input_tensor)
end_time = time.time()
# elapsed.append(end_time - start_time)
#print(detections['detection_scores'])

plt.rcParams['figure.figsize'] = [42, 21]
label_id_offset = 1
image_np_with_detections = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.int32),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=3,
      min_score_thresh=.50,
      agnostic_mode=False)
plt.subplot(2, 1, 0+1)
plt.imshow(image_np_with_detections)

In [ ]:
def make_predictions(image_path):
  image_np = load_image_into_numpy_array(image_path)
  #print(image_np)
  input_tensor = np.expand_dims(image_np, 0)
  start_time = time.time()
  detections = detect_fn(input_tensor)
  end_time = time.time()
  # elapsed.append(end_time - start_time)
  #print(detections['detection_scores'])

  plt.rcParams['figure.figsize'] = [42, 21]
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=3,
        min_score_thresh=.50,
        agnostic_mode=False)
  plt.subplot(2, 1, 0+1)
  plt.imshow(image_np_with_detections)

## EXAMPLE 1

In [ ]:
|# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True: 
    ret, frame = cap.read()
    print(frame)
    image_np = np.array(frame)
    print(image_np)
    # image_np = load_image_into_numpy_array(img)

    image_np_expanded = np.expand_dims(image_np, axis=0)

    detections = detect_fn(input_tensor) 
    output_dict = detections


    plt.rcParams['figure.figsize'] = [42, 21]
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          detections['detection_classes'][0].numpy().astype(np.int32),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=3,
          min_score_thresh=.50,
          agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

None
None


ValueError: ignored

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

## Helper Functions
Below are a few helper function to make converting between different image data types and formats. 

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
from PIL import Image as ImagePIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])

  image = ImagePIL.open(BytesIO(image_bytes))
  (im_width, im_height) = image.size
  image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img, image_np

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

## Haar Cascade Classifier
For this tutorial we will run a simple object detection algorithm called Haar Cascade on our images and video fetched from our webcam. OpenCV has a pre-trained Haar Cascade face detection model. 

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

## Webcam Videos
Running code on webcam video is a little more complex than images. We need to start a video stream using our webcam as input. Then we run each frame through our progam (face detection) and create an overlay image that contains bounding box of detection(s). We then overlay the bounding box image back onto the next frame of our video stream.




In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img, image_np = js_to_image(js_reply["img"])

    
    input_tensor = np.expand_dims(image_np, 0)
    start_time = time.time()
    detections = detect_fn(input_tensor)
    output_dict = detections
    end_time = time.time()
    # elapsed.append(end_time - start_time)
    # print(detections['detection_scores'])

    plt.rcParams['figure.figsize'] = [42, 21]
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          detections['detection_classes'][0].numpy().astype(np.int32),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=3,
          min_score_thresh=.50,
          agnostic_mode=False)
    # print(jpg_as_np)

    boxes = np.squeeze(detections['detection_boxes'][0].numpy())
    scores = np.squeeze(detections['detection_scores'][0].numpy().astype(np.int32))
    print(scores)
    #set a min thresh score, say 0.8
    min_score_thresh = 0.1
    bboxes = boxes[scores > min_score_thresh]

    #get image size
    im_width, im_height = 480,640
    final_box = []
    for box in bboxes:
        ymin, xmin, ymax, xmax = box
        final_box.append([xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height])

    # print(final_box)

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

## Hope You Enjoyed!
If you enjoyed the tutorial and want to see more videos or tutorials check out my YouTube channel [HERE](https://www.youtube.com/channel/UCrydcKaojc44XnuXrfhlV8Q?sub_confirmation=1)

Have a great day!

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
#from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  make_predictions(filename)
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))